## Attendance List Parser

This notebook looks for two files:

1. an `apprentices.csv` that contains the columns `name`, `email` and `cohort`, and lists **all** the apprentices in your cohort. This should not need to change once set up.
2. an `attendance.csv` that just contains the columns `Name` and `Email`, and just lists the apprentices that attended the workshop in question. This can be most easily obtained by just copying the attendance section out of the Teams report and pasting it into a new CSV.

See the included example CSVs for details - you can just copy and paste directly into these (don't forget to remove the example rows!)

Once both these files are in place, just run the code below. **Note:** To restrict your list to just one cohort, enter the cohort name exactly as it appears in the `apprentices.csv` file in the variable `COHORT` in the first cell.

In [1]:
import pandas as pd

# read in all apprentice names, emails and cohorts
apps = pd.read_csv('apprentices.csv')
apps['email'] = apps['email'].str.lower()
apps.rename(columns = {'name': 'app_name'}, inplace = True)

# filter for a certain cohort in your apprentices list
COHORT = 'DF(OPEN)-SEPT (TEAMS) E'
apps = apps[apps['cohort'] == COHORT].copy()

# read in attendance report data and just keep name and email columns
attn = pd.read_csv('attendance.csv')[['Name', 'Email']]
attn.columns = attn.columns.str.lower()
attn.rename(columns = {'name': 'teams_handle'}, inplace = True)
attn['email'] = attn['email'].str.lower()
attn['email'].fillna('no email', inplace = True)
attn['attended'] = 'yes'
attn.drop_duplicates(inplace = True)

# merge
all_attn = apps.merge(attn, on = ['email'], how = 'outer')
all_attn['attended'].fillna('no', inplace = True)

## Did Not Attend

The following apprentices did **not** attend:

In [2]:
all_attn[all_attn['attended'] == 'no'][['app_name', 'cohort']].sort_values(by = ['cohort', 'app_name'])

,app_name,cohort
1,Scam Likely,DF(OPEN)-SEPT (TEAMS) E


## Attended, No Email

The following apprentices attended but did not provide an email to Teams, and so need to be manually checked:

In [3]:
all_attn[all_attn['email'] == 'no email'][['teams_handle']]

,teams_handle
2,Scam (Likely)


## Attended

The following apprentices attended:

In [4]:
all_attn[all_attn['attended'] == 'yes'][['app_name', 'cohort']].dropna()

,app_name,cohort
0,John Smith,DF(OPEN)-SEPT (TEAMS) E
